In [1]:
import os
import numpy as np
import pandas as pd
import hickle

from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline


# Set working directory
DATA_DIR = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/data/'
# Set image directory which is off of GitHub
IMG_DIR_1 = 'C:/Users/thoma/Documents/00GitHub/00_LOCAL_ONLY/Cellavision_Trial_images/Test3_AltFebruary/'
IMG_DIR_2 = 'C:/Users/thoma/Pictures/HemePictures/'

WRONG_CSV = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/wrong_cell_csv/'
WRONG_CELLS = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/wrong_cells/'

In [2]:
# http://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script

import sys
sys.path.append('C:/Anaconda/Lib/site-packages')

import cv2
import PIL

In [15]:
# Set DATA directory
CSV_DIR = 'C:/Users/thoma/Documents/00GitHub/rbc_cnn/csv/'
path =r'C:\Users\thoma\Documents\00GitHub\rbc_cnn\csv' # use your path

def csv_to_dataFrame(path):
    
    # output csv
    #fout=open(outputFile,"a")
    i = 0 

    for root, dirs, files in os.walk(path):
        frame = pd.DataFrame()
        list_ = []
        for file_ in files:
            csv_path = os.path.join(root, file_)
            df = pd.read_csv(csv_path,index_col=0, header=0)
            list_.append(df)
        frame = pd.concat(list_)
    frame = frame.reset_index(drop=True)
    return frame

In [16]:
df = csv_to_dataFrame(CSV_DIR)

In [6]:
wrong = np.genfromtxt(WRONG_CSV+'wrong.csv', delimiter=',',dtype=int)
wrong

array([3479, 3003,  525,  716,  781, 2514,  507,  208,   67, 3576, 2321,
        616,  695, 1970, 2653, 3492,  789, 3627, 1735, 2687,   10,  388,
       3352, 1399,  794,  437,  514,  627])

In [6]:
GROUPS = {
        0:'NORMAL',
        1:'Echinocyte',
        2:'Dacrocyte',
        3:'Schistocyte',
        4:'Elliptocyte',
        5:'Acanthocyte',
        6:'Target cell',
        7:'Stomatocyte',
        8:'Spherocyte',
        9:'Overlap'
    }
wrong_df = pd.DataFrame.from_csv(WRONG_CELLS+'wrong_cells_september.csv',index_col=None)
for num in range(10):
    wrong_df = wrong_df.replace(to_replace=num, value=GROUPS[num])
wrong_df.head()

,correct_x,predicted_y,pk_value
0,Echinocyte,NORMAL,2251
1,Elliptocyte,Schistocyte,3557
2,NORMAL,Echinocyte,3534
3,Schistocyte,Acanthocyte,627
4,Dacrocyte,Schistocyte,1399


In [7]:
def wrong_image(dataframe):
    
    cropped_array = []
    i = 0
    
    for x in range(len(wrong_df)):

        correct_label = wrong_df.ix[x]['correct_x']
        wrong_label = wrong_df.ix[x]['predicted_y']
        pk_value = wrong_df.ix[x]['pk_value']
        
        cell_ix = df.ix[df['pk']==pk_value]
        img = df.get_value(cell_ix.index[0], 'image')
        path = "C:/Users/thoma/Documents/00GitHub/00_LOCAL_ONLY/00RbcCNN_Sln_Images/{}".format(img)
        
        im = cv2.imread(path)

        # im = cv2.imread(imgPath) 

        x = df.get_value(cell_ix.index[0], 'x')
        y = df.get_value(cell_ix.index[0], 'y')
        label = df.get_value(cell_ix.index[0], 'label')
        
        new_x = x - 35
        new_y = y - 35
        new_w = 70
        new_h = 70

        try:
            croppedCell = im[new_y:new_y+new_h, new_x:new_x+new_w]
            cropped_array.append(croppedCell)
            
            ## save each object
            cv2.imwrite(WRONG_CELLS+'september_1\{}_{}_{}.jpg'.format(correct_label,wrong_label,pk_value), croppedCell)
            
        except Exception, e:
            print e, img
    return cropped_array

    # pic = pic / 255        

In [17]:
wrong_cells = wrong_image(wrong_df)

'NoneType' object has no attribute '__getitem__' Targets - 762601381HV_001.jpg
'NoneType' object has no attribute '__getitem__' Schistocytes - 762501373HV_001.jpg
